In [3]:
# -*- coding: utf-8 -*-
# ============================================================
# ФИНАЛЬНЫЙ ПРОЕКТ: A/B-ТЕСТ РАЗМЕЩЕНИЯ РЕКЛАМНОГО БЛОКА
# ============================================================

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

# Настройки
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")
pd.set_option('display.max_columns', None)

# -------------------------------------------------
# 🔍 Автоматическое определение BASE_DIR
# -------------------------------------------------
current_dir = os.path.abspath(os.getcwd())
BASE_DIR = current_dir

max_levels = 5
found_data = False
for _ in range(max_levels):
    if os.path.exists(os.path.join(BASE_DIR, "Data")):
        found_data = True
        break
    parent_dir = os.path.dirname(BASE_DIR)
    if parent_dir == BASE_DIR:
        break
    BASE_DIR = parent_dir

if not found_data:
    BASE_DIR = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir))

DATA_RAW = os.path.join(BASE_DIR, "Data", "data_raw.csv")
FINAL_XLSX = os.path.join(BASE_DIR, "Data", "final_results_to_analyze.xlsx")
OUTPUT_MD = os.path.join(BASE_DIR, "qwen3_report.md")

print(f"📁 Текущая директория: {os.getcwd()}")
print(f"📁 Базовая директория: {BASE_DIR}")
print(f"📁 data_raw.csv: {DATA_RAW}")
print(f"📁 final_results_to_analyze.xlsx: {FINAL_XLSX}")

# Проверка файлов
assert os.path.exists(DATA_RAW), f"❌ Не найден: {DATA_RAW}"
assert os.path.exists(FINAL_XLSX), f"❌ Не найден: {FINAL_XLSX}"

# -------------------------------------------------
# 1. Обработка исторических данных (воронка)
# -------------------------------------------------
print("\n📥 Загрузка data_raw.csv...")
df_raw = pd.read_csv(DATA_RAW, parse_dates=['dt'])

# Нормализация event_type (на случай пробелов/регистра)
df_raw['event_type'] = df_raw['event_type'].astype(str).str.lower().str.strip()

# Диагностика
print("\n🔍 Уникальные значения event_type:", df_raw['event_type'].unique())
print("📊 Распределение событий:")
print(df_raw['event_type'].value_counts())

# ВАЖНО: в реальных данных — 'views', 'clicks', 'adds'
required_events = {'views', 'clicks', 'adds'}
actual_events = set(df_raw['event_type'])
missing_events = required_events - actual_events
if missing_events:
    raise ValueError(f"❌ Отсутствуют события: {missing_events}. Найдены: {actual_events}")

# Построение воронки на уровне (дата, пользователь)
funnel = df_raw.groupby(['dt', 'client_id', 'event_type']).size().unstack(fill_value=0).reset_index()
funnel.columns.name = None

# Убедимся, что все колонки есть
for event in required_events:
    if event not in funnel.columns:
        funnel[event] = 0

# Бинарные флаги: было ли событие хотя бы раз в день
funnel['has_view'] = (funnel['views'] > 0).astype(int)
funnel['has_click'] = (funnel['clicks'] > 0).astype(int)
funnel['has_add'] = (funnel['adds'] > 0).astype(int)

# Агрегация до пользователя
user_hist = funnel.groupby('client_id').agg(
    total_views=('has_view', 'sum'),
    total_clicks=('has_click', 'sum'),
    total_adds=('has_add', 'sum')
).reset_index()

# Пользователи с кликами — для CR_add
user_with_clicks = user_hist[user_hist['total_clicks'] > 0].copy()
if user_with_clicks.empty:
    raise ValueError("❌ Нет пользователей с кликами — невозможно рассчитать CR_add.")
user_with_clicks['cr_add'] = user_with_clicks['total_adds'] / user_with_clicks['total_clicks']

# Для CTR
user_with_views = user_hist[user_hist['total_views'] > 0].copy()
user_with_views['ctr'] = user_with_views['total_clicks'] / user_with_views['total_views']

# -------------------------------------------------
# 2. Расчёт MDE (дельта-метод для ratio-метрики)
# -------------------------------------------------
def var_ratio(x, y):
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    if mean_y == 0:
        return np.nan
    n = len(x)
    if n < 2:
        return 0.0
    var_x = np.var(x, ddof=1)
    var_y = np.var(y, ddof=1)
    cov_xy = np.cov(x, y, ddof=1)[0, 1]
    return var_x / mean_y**2 + var_y * mean_x**2 / mean_y**4 - 2 * mean_x * cov_xy / mean_y**3

x = user_with_clicks['total_adds'].values
y = user_with_clicks['total_clicks'].values
mu_cr = np.mean(user_with_clicks['cr_add'])
std_cr = np.sqrt(var_ratio(x, y))

if np.isnan(mu_cr) or np.isnan(std_cr) or std_cr <= 0:
    raise ValueError("❌ Невозможно рассчитать MDE: недостаточно данных.")

def get_MDE(mu, std, n_total, alpha=0.05, beta=0.2):
    t_alpha = stats.norm.ppf(1 - alpha / 2)
    t_beta = stats.norm.ppf(1 - beta)
    n_per_group = n_total / 2
    mde_abs = (t_alpha + t_beta) * std * np.sqrt(2 / n_per_group)
    return mde_abs / mu * 100

sample_sizes = [20_000, 40_000, 60_000, 80_000, 100_000]
mde_values = []
for n in sample_sizes:
    try:
        mde_val = get_MDE(mu_cr, std_cr, n)
        mde_values.append(round(mde_val, 2))
    except:
        mde_values.append(np.nan)

mde_df = pd.DataFrame({
    'Общая выборка': sample_sizes,
    'MDE (%)': mde_values
})

valid = mde_df.dropna().query("`MDE (%)` <= 2.5")
chosen_n = int(valid.iloc[0]['Общая выборка']) if not valid.empty else int(mde_df.iloc[-1]['Общая выборка'])
chosen_mde = mde_df[mde_df['Общая выборка'] == chosen_n].iloc[0]['MDE (%)']

# -------------------------------------------------
# 3. Загрузка результатов A/B-теста
# -------------------------------------------------
print("\n📥 Загрузка final_results_to_analyze.xlsx...")
df_res = pd.read_excel(FINAL_XLSX)

# Агрегация по пользователю
user_ab = df_res.groupby('client_id').agg(
    ab_group=('ab_group', 'first'),
    has_click=('is_view_ads', 'max'),
    has_add=('is_adds_ads', 'max'),
    adds_revenue=('sum_adds_ads', 'sum'),
    orders_count=('cnt_orders_ads', 'sum'),
    adds_count=('cnt_adds_ads', 'sum')
).reset_index()

# Агрегация по группам
metrics = user_ab.groupby('ab_group').agg(
    users=('client_id', 'count'),
    clicks=('has_click', 'sum'),
    adds=('has_add', 'sum'),
    revenue_adds=('adds_revenue', 'sum'),
    orders=('orders_count', 'sum'),
    adds_total=('adds_count', 'sum')
).reset_index()

metrics['CR_add'] = metrics['adds'] / metrics['clicks']
metrics['ARPU'] = metrics['revenue_adds'] / metrics['users']
metrics['CR_order'] = metrics['orders'] / metrics['clicks']
metrics['Avg_check_add'] = metrics['revenue_adds'] / metrics['adds_total'].replace(0, np.nan)
metrics = metrics.fillna(0)

# -------------------------------------------------
# 4. Определение групп
# -------------------------------------------------
groups = sorted(metrics['ab_group'].unique())
if len(groups) < 2:
    raise ValueError(f"❌ Нужны 2 группы, найдена только: {groups}")

control_name, treatment_name = groups[0], groups[1]
control = metrics[metrics['ab_group'] == control_name].iloc[0]
treatment = metrics[metrics['ab_group'] == treatment_name].iloc[0]

print(f"✅ Группы: контроль = '{control_name}', тест = '{treatment_name}'")

cr_diff_abs = treatment['CR_add'] - control['CR_add']
cr_diff_rel = cr_diff_abs / control['CR_add'] * 100 if control['CR_add'] > 0 else 0

# Статистика
count = np.array([control['adds'], treatment['adds']])
nobs = np.array([control['clicks'], treatment['clicks']])
if nobs[0] > 0 and nobs[1] > 0:
    _, p_val = proportions_ztest(count, nobs, alternative='larger')
    is_significant = p_val < 0.05
else:
    p_val = np.nan
    is_significant = False

# -------------------------------------------------
# 5. Визуализации
# -------------------------------------------------
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

sns.histplot(user_with_clicks['cr_add'], bins=50, kde=True, ax=axes[0,0])
axes[0,0].set_title('CR_add до эксперимента')

sns.histplot(user_with_views['ctr'], bins=50, kde=True, ax=axes[0,1])
axes[0,1].set_title('CTR до эксперимента')

sns.barplot(data=metrics, x='ab_group', y='CR_add', ax=axes[1,0])
axes[1,0].set_title(f'CR_add по группам\nΔ = +{cr_diff_rel:.1f}%')
for p in axes[1,0].patches:
    axes[1,0].annotate(f"{p.get_height():.3f}", (p.get_x() + p.get_width()/2., p.get_height()),
                       ha='center', va='bottom', fontsize=10)

sns.barplot(data=metrics, x='ab_group', y='ARPU', ax=axes[1,1])
axes[1,1].set_title('ARPU')
for p in axes[1,1].patches:
    axes[1,1].annotate(f"₽{p.get_height():.1f}", (p.get_x() + p.get_width()/2., p.get_height()),
                       ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plot_path = os.path.join(BASE_DIR, "ab_test_results_qwen3.png")
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
plt.close()

# -------------------------------------------------
# 6. Генерация отчёта
# -------------------------------------------------
def df_to_md(df):
    try:
        from tabulate import tabulate
        return tabulate(df, headers='keys', tablefmt='pipe', showindex=False, floatfmt=".3f")
    except ImportError:
        return df.to_string(index=False)

mde_md = df_to_md(mde_df)
metrics_md = df_to_md(metrics[['ab_group', 'users', 'clicks', 'adds', 'CR_add', 'ARPU', 'CR_order']])

md_content = f"""# 📊 Отчёт по А/Б-тесту: Перемещение рекламного блока

## 🔑 Выводы
- **Ключевая метрика**: CR_add (конверсия в добавление = adds / clicks)
- **Группы**: контроль = `{control_name}`, тест = `{treatment_name}`
- **Результат**: {"✅ Успех" if is_significant and cr_diff_rel > 0 else "❌ Без эффекта"}
  - CR_add: **{control['CR_add']:.3f} → {treatment['CR_add']:.3f}**
  - Рост: **+{cr_diff_rel:.1f}%**
  - p-value: **{p_val:.4f}** → {"значимо" if is_significant else "не значимо"}
- **Рекомендация**: {"🚀 Внедрить" if is_significant and cr_diff_rel > 0 else "🛑 Не внедрять"}

## ⚙️ Дизайн
- Историческая CR_add: {mu_cr:.3f}
- Выборка: {chosen_n:,} пользователей
- MDE: {chosen_mde:.2f}%

### MDE vs. выборка
{mde_md}

## 📈 Метрики по группам
{metrics_md}

### Дополнительно
- Средний чек (контроль): ₽{control['Avg_check_add']:.2f}
- Средний чек (тест): ₽{treatment['Avg_check_add']:.2f}
- CR в заказ (контроль): {control['CR_order']:.3f}
- CR в заказ (тест): {treatment['CR_order']:.3f}

## 📎 Графики
![Результаты](ab_test_results_qwen3.png)

## 💡 Заключение
{"Изменение позиции рекламы **значимо улучшает** метрики." if is_significant and cr_diff_rel > 0 else "Эффект не подтверждён. Не рекомендуется внедрять."}
"""

with open(OUTPUT_MD, 'w', encoding='utf-8') as f:
    f.write(md_content)

print(f"\n✅ Отчёт сохранён: {OUTPUT_MD}")
print(f"📊 График сохранён: {plot_path}")

📁 Текущая директория: c:\ZC\finish\notebooks
📁 Базовая директория: c:\ZC\finish
📁 data_raw.csv: c:\ZC\finish\Data\data_raw.csv
📁 final_results_to_analyze.xlsx: c:\ZC\finish\Data\final_results_to_analyze.xlsx

📥 Загрузка data_raw.csv...

🔍 Уникальные значения event_type: ['clicks' 'adds' 'views']
📊 Распределение событий:

🔍 Уникальные значения event_type: ['clicks' 'adds' 'views']
📊 Распределение событий:
event_type
views     27006473
clicks      535927
adds        213648
Name: count, dtype: int64
event_type
views     27006473
clicks      535927
adds        213648
Name: count, dtype: int64

📥 Загрузка final_results_to_analyze.xlsx...

📥 Загрузка final_results_to_analyze.xlsx...
✅ Группы: контроль = 'control', тест = 'test'
✅ Группы: контроль = 'control', тест = 'test'

✅ Отчёт сохранён: c:\ZC\finish\qwen3_report.md
📊 График сохранён: c:\ZC\finish\ab_test_results_qwen3.png

✅ Отчёт сохранён: c:\ZC\finish\qwen3_report.md
📊 График сохранён: c:\ZC\finish\ab_test_results_qwen3.png
